In [1]:
import requests
import codecs
import csv
from bs4 import BeautifulSoup 
import json
import io
import gzip
from html.parser import HTMLParser
import nltk
import re
import collections
from collections import defaultdict
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/manasikulkarni/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
get_requests1 = requests.get('http://index.commoncrawl.org/CC-MAIN-2019-09-index?url=http%3A%2F%2Fhttps://www.bleacherreport.com/&output=json')
get_requests2 = requests.get('http://index.commoncrawl.org/CC-MAIN-2019-04-index?url=http%3A%2F%2Fhttps://www.bleacherreport.com/&output=json')
get_requests3 = requests.get('http://index.commoncrawl.org/CC-MAIN-2019-13-index?url=http%3A%2F%2Fhttps://www.sportingnews.com/&output=json')
requestsArray = []
requestsArray.append(get_requests1)
requestsArray.append(get_requests2)
requestsArray.append(get_requests3)

In [3]:
pageList = []
for r in requestsArray:
    if r.status_code == 200:
        pages = r.content.splitlines()
        for i in pages:
            pageList.append(json.loads(i))

print(pages)


[b'{"urlkey": "com,sportingnews)/", "timestamp": "20190319153722", "digest": "ULN6WICTL6EEZXBXC6ASRKL2I2C2W24K", "offset": "363158946", "length": "39079", "url": "http://www.sportingnews.com/", "languages": "eng", "filename": "crawl-data/CC-MAIN-2019-13/segments/1552912201996.61/warc/CC-MAIN-20190319143502-20190319165502-00131.warc.gz", "mime": "text/html", "charset": "UTF-8", "status": "200", "mime-detected": "text/html"}', b'{"urlkey": "com,sportingnews)/", "timestamp": "20190319214259", "digest": "UKQJ3PP7WZBADFY66DQXHLCL4H3BMQJ7", "offset": "2304622", "length": "38890", "url": "http://www.sportingnews.com", "filename": "crawl-data/CC-MAIN-2019-13/segments/1552912202131.54/robotstxt/CC-MAIN-20190319203912-20190319225912-00556.warc.gz", "mime": "text/html", "status": "200", "mime-detected": "text/html"}', b'{"urlkey": "com,sportingnews)/", "timestamp": "20190319214343", "digest": "UKQJ3PP7WZBADFY66DQXHLCL4H3BMQJ7", "offset": "372052403", "length": "38890", "url": "http://www.sporting

In [59]:
page = pageList[1]
print(page)

{'urlkey': 'com,bleacherreport)/', 'timestamp': '20190216012333', 'digest': '3I42H3S6NNFQ2MSVX7XZKYAYSCX5QBYJ', 'offset': '919508', 'length': '587', 'url': 'http://bleacherreport.com/', 'filename': 'crawl-data/CC-MAIN-2019-09/segments/1550247479729.27/crawldiagnostics/CC-MAIN-20190216004609-20190216030609-00383.warc.gz', 'mime': 'unk', 'status': '301', 'mime-detected': 'application/octet-stream'}


In [4]:
for page in pageList:
    offset, length = int(page['offset']), int(page['length'])
    offset_end = offset + length - 1
    print(offset, length, offset_end)

    #prefix = 'https://aws-publicdatasets.s3.amazonaws.com/'
    prefix = 'https://commoncrawl.s3.amazonaws.com/'

    response = requests.get(prefix + page['filename'], headers={'Range': 'bytes={}-{}'.format(offset, offset_end)})
    #response = requests.get("https://commoncrawl.s3.amazonaws.com/crawl-data/CC-MAIN-2019-09/segments/1550247479627.17/wet/CC-MAIN-20190215224408-20190216010408-00271.warc.wet.gz", headers={'Range': 'bytes={}-{}'.format(offset, offset_end)})
    print(response)
    
    raw_data = io.BytesIO(response.content)
    print(raw_data)
    f = gzip.GzipFile(fileobj=raw_data)
    print(f)
    
    data = f.read()
    doc = BeautifulSoup(io.BytesIO(data), 'lxml')

    all_a = doc.find_all('a')
    hrefList = []
    if all_a:
        for l in all_a:
            #print(l)
            href = l.attrs.get('href')

            if href is not None:
                if href.startswith('http'):
                    hrefList.append(href)
                    
with codecs.open('breacher_news_links.csv', 'wb', encoding = 'utf8') as output:
    f = ['url']
    write_csv = csv.DictWriter(output, fieldnames = f)
    write_csv.writeheader()

    for h in hrefList:
        write_csv.writerow({'url':h})
        
hrefList = list( dict.fromkeys(hrefList) )

for h in hrefList:
    print(h)

513826052 112788 513938839
<Response [206]>
<gzip _io.BytesIO object at 0x1a19b6d728 0x1a193ca358>
919508 587 920094
<Response [206]>
<gzip _io.BytesIO object at 0x1a19b6d5c8 0x1a19ba1a58>
478945419 116238 479061656
<Response [206]>
<gzip _io.BytesIO object at 0x1a19b6d728 0x1a1a46da20>
955469 591 956059
<Response [206]>
<gzip _io.BytesIO object at 0x1a1a464a40 0x1a19c1cd68>
496154076 112503 496266578
<Response [206]>
<gzip _io.BytesIO object at 0x1a19b6d728 0x1a1a46da58>
1836300 111378 1947677
<Response [206]>
<gzip _io.BytesIO object at 0x1a1a7a70a0 0x1a19d6b9e8>
490484212 110776 490594987
<Response [206]>
<gzip _io.BytesIO object at 0x1a1aac6a98 0x1a1aadd2e8>
495628622 115589 495744210
<Response [206]>
<gzip _io.BytesIO object at 0x1a1aecc6d0 0x1a1aed67f0>
1777831 115417 1893247
<Response [206]>
<gzip _io.BytesIO object at 0x1a1b2b2c50 0x1a1b2c15c0>
498477814 115416 498593229
<Response [206]>
<gzip _io.BytesIO object at 0x1a1b6f2830 0x1a1b6f9cc0>
487113663 116409 487230071
<Response

<Response [206]>
<gzip _io.BytesIO object at 0x1a1a9971a8 0x1a1a9a1ef0>
363353382 39125 363392506
<Response [206]>
<gzip _io.BytesIO object at 0x1a1b31cca8 0x1a1b2f87b8>
363867235 38899 363906133
<Response [206]>
<gzip _io.BytesIO object at 0x1a1a3c3518 0x1a1a3eb828>
355570414 39131 355609544
<Response [206]>
<gzip _io.BytesIO object at 0x1a19dbefc0 0x104d3deb8>
355038646 39252 355077897
<Response [206]>
<gzip _io.BytesIO object at 0x104d71990 0x104d66908>
347628614 39275 347667888
<Response [206]>
<gzip _io.BytesIO object at 0x1a1a906570 0x1a1a8fdfd0>
1809644 39376 1849019
<Response [206]>
<gzip _io.BytesIO object at 0x1a19dbeeb8 0x1a1aadd748>
373547189 39510 373586698
<Response [206]>
<gzip _io.BytesIO object at 0x1a1c38cca8 0x1a1a0feda0>
http://www.sportingnews.com/us
http://www.sportingnews.com/us/ncaa-basketball/tournament/2019/bracket
http://www.sportingnews.com/us/nfl
http://www.sportingnews.com/us/nba
http://www.sportingnews.com/us/mlb
http://www.sportingnews.com/us/nhl
http://

In [5]:
import pandas as pd
df = pd.read_csv("breacher_news_links.csv")
url_list = df['url']

In [6]:
unique_url = set(url_list)
print(len(unique_url))

all_text = ""
filename = "common_crawl_article_breacher_links"
file_counter = 0
sno = nltk.wordnet.WordNetLemmatizer()

for urli in unique_url:
     if "videos" in urli or "photos" in urli or "instagram" in urli or "streamja" in urli or "facebook" in urli or "bit" in urli or "twitter" in urli:
        continue
        
     response = requests.get(urli)
     all_text = ""
     if response.status_code == 200:
         doc = BeautifulSoup(response.text, 'html.parser')
         name_box = doc.find_all('p')

         for name in name_box:
            for x in name.find_all('a'):
                x.replaceWithChildren()

         for link in name_box:
            link_text = link.text
   
            link_text = re.sub('\ by', '', link_text)
            link_text = re.sub('Advertisement  ', '', link_text)
            link_text = re.sub('Supported', '', link_text)
            link_text = re.sub('Advertisement$', '', link_text)

            filtered_text = re.sub('[0-9]+', ',', link_text)
            filtered_text = re.sub(r'[^\w\s]',' ',filtered_text)
            filtered_text = filtered_text.lower()

            inv_indx = defaultdict(list)

            words = (filtered_text.split())
            for word in words:
                inv_indx[word].append(1)

            #for word in inv_indx:
                #stem_word = sno.lemmatize(word)
                #if stem_word != word:
                    #filtered_text = filtered_text.replace(word,stem_word)

            all_text += filtered_text
            all_text+=" <BRK> "


         if all_text == "" or all_text == "<BRK>":
            continue

         file_counter += 1    
         curr_file_name = filename + str(file_counter) + ".txt"
         text_file = open(curr_file_name, "w")
         text_file.write( all_text)
         text_file.close()
         print("New file writing",file_counter)


111
New file writing 1
New file writing 2
New file writing 3
New file writing 4
New file writing 5
New file writing 6
New file writing 7
New file writing 8
New file writing 9
New file writing 10
New file writing 11
New file writing 12
New file writing 13
New file writing 14
New file writing 15
New file writing 16
New file writing 17
New file writing 18
New file writing 19
New file writing 20
New file writing 21
New file writing 22
New file writing 23
New file writing 24
New file writing 25
New file writing 26
New file writing 27
New file writing 28
New file writing 29
New file writing 30
New file writing 31
New file writing 32
New file writing 33
